In [3]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import utils

In [66]:
numpy_input_fn = tf.estimator.inputs.numpy_input_fn

X, Y, test_X, test_Y, classes = utils.load_dataset()
X = X.reshape(X.shape[0], -1) / 255.
Y = Y.reshape(-1)
test_X = test_X.reshape(test_X.shape[0], -1) / 255.
test_Y = test_Y.reshape(-1)

learning_rate = 0.0005

In [129]:
feature_x = tf.feature_column.numeric_column('feature_x', shape=(X.shape[1]))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

estimator = tf.estimator.DNNClassifier(
  feature_columns=[feature_x], hidden_units=[16], optimizer=optimizer, n_classes=2)

input_fn_train = numpy_input_fn(x={'feature_x': X}, y=Y, batch_size=64, shuffle=True, num_epochs=300)
input_fn_train_1 = numpy_input_fn(x={'feature_x': X}, y=Y, batch_size=64, shuffle=False)
input_fn_test = numpy_input_fn(x={'feature_x': test_X}, y=test_Y, batch_size=64, shuffle=False)

estimator.train(input_fn_train)
print('##################################')
estimator.evaluate(input_fn_train_1)
print('##################################')
estimator.evaluate(input_fn_test)

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/var/folders/lj/b2trvfyj0mjbjf90yx92y6d80000gn/T/tmpgecyrpk4', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x124f6f588>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 1 into /var/folders/lj/b2trvfyj0mjbjf90yx92y6d8000

{'accuracy': 0.72,
 'accuracy_baseline': 0.66,
 'auc': 0.7834224,
 'auc_precision_recall': 0.82436293,
 'average_loss': 0.9487519,
 'global_step': 980,
 'label/mean': 0.66,
 'loss': 47.437595,
 'prediction/mean': 0.44978404}

In [165]:
print(X[0], Y[0])
print(X[2], Y[2])
with tf.Graph().as_default():
  tf.set_random_seed(5)
  session = tf.InteractiveSession()
  
  X_ph = tf.placeholder(dtype=tf.float32, shape=(None, X.shape[1]))
  Y_ph = tf.placeholder(dtype=tf.float32, shape=(None, 1))
  
  W1 = tf.get_variable('W1', shape=(X.shape[1], 16), dtype=tf.float32, initializer=tf.truncated_normal_initializer())
  b1 = tf.get_variable('b1', shape=(16), dtype=tf.float32, initializer=tf.zeros_initializer())
  
  X_ = tf.nn.relu(tf.matmul(X_ph, W1) + b1)
  
  W2 = tf.get_variable('W2', shape=(16, 1), dtype=tf.float32, initializer=tf.truncated_normal_initializer())
  b2 = tf.get_variable('b2', shape=(1), dtype=tf.float32, initializer=tf.zeros_initializer())
  
  A = tf.matmul(X_, W2) + b2
  
  loss = tf.nn.sigmoid_cross_entropy_with_logits(logits=A, labels=Y_ph)
  loss = tf.reduce_mean(loss)
  accuracy = tf.reduce_sum(
    tf.cast(tf.equal(tf.greater(tf.sigmoid(A), 0.5), tf.greater(Y_ph, 0.5)), dtype=tf.float32)) / tf.cast(tf.shape(X_ph)[0], dtype=tf.float32)
  
  train_op = tf.train.GradientDescentOptimizer(learning_rate=0.001).minimize(loss)
  
  session.run(tf.global_variables_initializer())

  num_iterations = 500
  for i in range(num_iterations):
    _, loss_num = session.run([train_op, loss], feed_dict={X_ph: X, Y_ph: Y[:, np.newaxis]})
    
    if i % 50 == 0:
      accuracy_num = session.run(accuracy, feed_dict={X_ph: X, Y_ph: Y[:, np.newaxis]})
      print('training loss: {}, accuracy: {}'.format(loss_num, accuracy_num))
      accuracy_num = session.run(accuracy, feed_dict={X_ph: test_X, Y_ph: test_Y[:, np.newaxis]})
      print('testing accuracy: {}'.format(accuracy_num))



[0.06666667 0.12156863 0.21960784 ... 0.         0.         0.        ] 0
[0.32156863 0.27843137 0.26666667 ... 0.54117647 0.55294118 0.55686275] 1
training loss: 13.40841293334961, accuracy: 0.6076555252075195
testing accuracy: 0.2800000011920929
training loss: 4.891404628753662, accuracy: 0.540669858455658
testing accuracy: 0.5400000214576721
training loss: 3.4552433490753174, accuracy: 0.5693780183792114
testing accuracy: 0.5400000214576721
training loss: 2.8382580280303955, accuracy: 0.5741626620292664
testing accuracy: 0.5600000023841858
training loss: 2.4748759269714355, accuracy: 0.5885167717933655
testing accuracy: 0.5799999833106995
training loss: 2.207217216491699, accuracy: 0.5933014154434204
testing accuracy: 0.5799999833106995
training loss: 1.9726080894470215, accuracy: 0.5933014154434204
testing accuracy: 0.5799999833106995
training loss: 1.7954493761062622, accuracy: 0.5980861186981201
testing accuracy: 0.6000000238418579
training loss: 1.618738055229187, accuracy: 0.60